In [140]:
import numpy as np
from scipy import stats
from numpy.linalg import inv
from sigmoid import sigmoid
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression

In [144]:
y = 1.0 * np.random.choice(2, size = 200)
mvn = stats.multivariate_normal.rvs
X = np.array([mvn([1, 1] if heads else [4, 4]) for heads in y])

In [138]:
def fit(X, y):
    N, D = X.shape
    w = np.zeros(D)
    ybar = np.mean(y)
    w0 = np.log(ybar / (1 - ybar))

    for k in range(2):
        eta = w0 + X.dot(w)
        mu = sigmoid(eta)
        s = mu * (1 - mu)
        z = eta + (y - mu) / s
        S = np.diag(s)
        w = inv(X.T.dot(S).dot(X)).dot(X.T).dot(z)    
    return w0, w

def predict(model, X):
    w0, w = model
    return (sigmoid(w0 + X.dot(w)) > 0.5) * 1

In [136]:
def fit(X, y):
    N, D = X.shape
    
    def NLL(w):
        muw = mu(w)
        return -sum(y * np.log(muw) + (1 - y) * np.log(1 - muw))

    mu = lambda w: sigmoid(X.dot(w))
    jac = lambda w: X.T.dot(mu(w) - y)
    S = lambda w, mu: np.diag(mu * (1 - mu))
    hess = lambda w: X.T.dot(S(w, mu(w))).dot(X)

    w = np.ones(D)
    return minimize(NLL, w, jac = jac, hess = hess, method = 'Newton-CG').x

def predict(model, X):
    w = model
    X.dot(w)
    return (sigmoid(X.dot(w)) > 0.5) * 1

In [145]:
def fit(X, y):
    clf = LinearRegression()
    clf.fit(X, y)
    return clf

def predict(model, X):
    return (clf.predict(X) > 0) * 1

In [146]:
np.mean(y == predict(fit(X, y), X))

0.69499999999999995